In [1]:
import numpy as np
from scipy.special import hyp1f1
import pandas as pd

A Note on Units:
- Length in **bohrs** (0.529 angstroms)
- Energy in **hartrees** (27.2 eV)

In [2]:
# Basis functions
def eval_gaussian(r, R, alpha):
    '''
    Evaluates normalized Gaussian function at 3D location r with 3D mean R and 1D inverse-variance alpha. 
    '''
    return (2 * alpha / np.pi)**0.75 * np.exp(-alpha * np.dot(r - R, r - R))

In [3]:
def load_basis_set(filename='STO-3G.txt'):
    basis_dict = {}
    with open(filename, 'r') as fp:
        line = fp.readline()
        while line:
            if line[0] == '-':
                atom = line.split()[0][1:]
                orbital_types = []
                expos = []
                coefs = []
                line = fp.readline()
                while line[0] != '*':
                    orbital = line.split()[0]
                    if orbital == 'S':
                        orb_expos = []
                        orb_coefs = []
                        for _ in range(3):
                            line = fp.readline()
                            vals = [float(x) for x in list(line.split())]
                            orb_expos.append(vals[0])
                            orb_coefs.append(vals[1])
                        orbital_types.append('S')
                        expos.append(orb_expos)
                        coefs.append(orb_coefs)
                    elif orbital == 'SP':
                        orb_expos = []
                        orb_coefs_S = []
                        orb_coefs_P = []
                        for _ in range(3):
                            line = fp.readline()
                            vals = [float(x) for x in list(line.split())]
                            orb_expos.append(vals[0])
                            orb_coefs_S.append(vals[1])
                            orb_coefs_P.append(vals[2])
                        orbital_types.append('S')
                        expos.append(orb_expos)
                        coefs.append(orb_coefs_S)
                        for t in ['Px', 'Py', 'Pz']:
                            orbital_types.append(t)
                            expos.append(orb_expos)
                            coefs.append(orb_coefs_P)
                    else:
                        raise ValueError('Orbital {} not supported!'.format(orbital))
                    line = fp.readline()
                basis_dict[atom] = {'orbital_types' : np.array(orbital_types), 
                                    'expos' : np.array(expos),
                                    'coefs': np.array(coefs)}
            line = fp.readline()
        return basis_dict

In [4]:
basis_set = load_basis_set()

In [5]:
# Rs = np.array([[0, 0, 0], [1.4, 0, 0]])
# atoms = np.array(['H', 'H'])
# Zs = np.array([1, 1])
# Rs = np.array([[0, 0, 0], [1.4632, 0, 0]])

# Rs = np.array([
#         [-0.422389457, -0.422389457, -0.422389457],
#         [+0.422389457, +0.422389457, +0.422389457]
#     ])
# atoms = np.array(['He', 'H'])
# Zs = np.array([2, 1])

# Rs = np.array([
#         [0, 0, 0],
#         [2.074, 0, 0]
#     ])
# atoms = np.array(['N', 'N'])
# Zs = np.array([7, 7])

###################################
############## WATER ##############
###################################
# Rs = np.array([
#         [0.000000000000,  -0.143225816552,   0.000000000000],
#         [1.638036840407,   1.136548822547,  -0.000000000000],
#         [-1.638036840407,   1.136548822547,  -0.000000000000]
#     ])
# atoms = np.array(['O', 'H', 'H'])
# Zs = np.array([8, 1, 1])

###################################
############ METHANE ##############
###################################
Rs = np.array([
        [-0.000000000000,   0.000000000000,   0.000000000000],
        [1.183771681898,  -1.183771681898,  -1.183771681898],
        [1.183771681898,   1.183771681898,   1.183771681898],
        [-1.183771681898,   1.183771681898,  -1.183771681898],
        [-1.183771681898,  -1.183771681898,   1.183771681898]
])
atoms = np.array(['C', 'H', 'H', 'H', 'H'])
Zs = np.array([6, 1, 1, 1, 1])

n_nuclei = atoms.shape[0]

ao_types = [basis_set[a]['orbital_types'] for a in atoms]
ao_exps = [basis_set[a]['expos'] for a in atoms]
ao_coefs = [basis_set[a]['coefs'] for a in atoms]

orbital_counts = [len(at) for at in ao_types]
ao_types = np.hstack(ao_types)
ao_exps = np.vstack(ao_exps)
ao_coefs = np.vstack(ao_coefs)
ao2nuc = np.repeat(np.arange(n_nuclei), orbital_counts, axis=0)
n_orbitals = ao2nuc.shape[0]
N = Zs.sum()

In [6]:
# N = 2
# ao_exps[0] = ao_exps[1] / (1.24 ** 2) * (2.0925 ** 2) 

In [7]:
def overlap_expansion_coefs(a, b, R_A, R_B, l_A_max=0, l_B_max=0):
    '''
    Recursively compute expansion coefficients for the overlap distribution of two Cartesian Gaussians expressed as Hermite Gaussians.  Inputs Gaussian exponents `a`, `b`, centers `R_A`, `R_B`, and maximum azimuthal quantum numbers `l_A_max`, `l_b_max`.  Outputs a (3, t_max+1, l_A_max+1, l_B_max+1) tensor `E` where t_max = l_A_max + l_B_max.  
    '''
    t_max = l_A_max + l_B_max + 1
    E = np.full((3, t_max + 1, l_A_max + 1, l_B_max + 1), np.nan)
    
    for i in range(3):
        
        p = a + b
        R_P_i = (a * R_A[i] + b * R_B[i]) / p
        R_AB_i = R_B[i] - R_A[i]
        R_AP_i = R_P_i - R_A[i]
        R_BP_i = R_P_i - R_B[i]
        
        # Base Case (l_A = 0)
        E[i, 0, 0, 0] = np.exp(-a * b / p * R_AB_i ** 2)
        E[i, 1:, 0, 0] = 0
        
        E[i, t_max, :, :] = 0
        
        for l_A in range(1, l_A_max + 1):
            for t in range(0, t_max):
                if t > l_A:
                    E[i, t, l_A, 0] = 0
                else:
                    E[i, t, l_A, 0] = 1 / (2*p) * E[i, t-1, l_A-1, 0] + \
                                        R_AP_i * E[i, t, l_A-1, 0] + \
                                        (t+1) * E[i, t+1, l_A-1, 0]
        
        for l_B in range(1, l_B_max + 1):
            for l_A in range(0, l_A_max + 1):
                for t in range(0, t_max):
                    E[i, t, l_A, l_B] = 1 / (2*p) * E[i, t-1, l_A, l_B-1] + \
                                        R_BP_i * E[i, t, l_A, l_B-1] + \
                                        (t+1) * E[i, t+1, l_A, l_B-1]                
    return E

In [8]:
def angular_vector(ao_type):
    '''
    Matches the input string `ao_type` to a vector of 3D Cartesian coordinates representing the azimuthal and magnetic quantum numbers.  Currently, only s and p type orbitals are supported.
    '''
    if ao_type == 'S':
        return np.array([0, 0, 0])
    elif ao_type == 'Px':
        return np.array([1, 0, 0])
    elif ao_type == 'Py':
        return np.array([0, 1, 0])
    elif ao_type == 'Pz':
        return np.array([0, 0, 1])
    else:
        raise NotImplementedError()

def gaussian_norm_coef(a, l_A):
    '''
    Calculates the normalization constant for a Gaussian given exponent `a` and orbital type `l_A`.
    '''
    if l_A == 0:
        return (2 * a / np.pi) ** (3/4)
    elif l_A == 1:
        return (128 * a ** 5 / np.pi ** 3) ** (1/4)
    else:
        raise NotImplementedError('Input `l_A` must be an integer <= 1.')
        
        
def overlap(E, a, b, L_A, L_B):
    '''
    Computes the overlap integral given expansion coefficients `E` and parameters for two Gaussian. 
    '''
    p = a + b
    n_A = gaussian_norm_coef(a, L_A.sum())
    n_B = gaussian_norm_coef(b, L_B.sum())
    S_x = (np.pi / p) ** (1/2) * E[0, 0, L_A[0], L_B[0]]
    S_y = (np.pi / p) ** (1/2) * E[1, 0, L_A[1], L_B[1]]
    S_z = (np.pi / p) ** (1/2) * E[2, 0, L_A[2], L_B[2]]
    S = n_A * n_B * S_x * S_y * S_z
    return S

def _kinetic_1d(E, a, b, l_A, l_B):
    '''
    Computes the kinetic energy integral in one dimension given expansion coefficients `E` and parameters for two Gaussian. 
    '''
    p = a + b
    S_i_jp2 = (np.pi / p) ** (1/2) * E[0, l_A, l_B + 2]
    S_i_j = (np.pi / p) ** (1/2) * E[0, l_A, l_B]
    if l_B >= 2:
        S_i_jm2 = (np.pi / p) ** (1/2) * E[0, l_A, l_B - 2]
    else:
        S_i_jm2 = 0
    T = -2 * b ** 2 * S_i_jp2 + b * (2 * l_B + 1) * S_i_j - 1/2 * l_B * (l_B - 1) * S_i_jm2
    return T

def kinetic(E, a, b, L_A, L_B):
    '''
    Computes the kinetic energy integral given expansion coefficients `E` and parameters for two Gaussian. 
    '''
    p = a + b
    n_A = gaussian_norm_coef(a, L_A.sum())
    n_B = gaussian_norm_coef(b, L_B.sum())
    S_x = (np.pi / p) ** (1/2) * E[0, 0, L_A[0], L_B[0]] 
    S_y = (np.pi / p) ** (1/2) * E[1, 0, L_A[1], L_B[1]] 
    S_z = (np.pi / p) ** (1/2) * E[2, 0, L_A[2], L_B[2]]
    T_x = _kinetic_1d(E[0], a, b, L_A[0], L_B[0])
    T_y = _kinetic_1d(E[1], a, b, L_A[1], L_B[1])
    T_z = _kinetic_1d(E[2], a, b, L_A[2], L_B[2])
    return n_A * n_B * (T_x * S_y * S_z + S_x * T_y * S_z + S_x * S_y * T_z)

In [9]:
def boys(n, T):
    return hyp1f1(n + 0.5, n + 1.5, -T) / (2.0 * n + 1.0) 

In [10]:
def coulomb_auxiliary_integrals(p, R_P, R_C, t_max=1, u_max=1, v_max=1):
    
    R_PC = R_P - R_C
    R2_PC = np.sum(R_PC ** 2)
    
    n_max = t_max + u_max + v_max
    R = np.full((n_max + 1, t_max + 1, u_max + 1, v_max + 1), np.nan)
    
    # Base Case (n, t=0, u=0, v=0)
    for n in range(0, n_max + 1):
        R[n, 0, 0, 0] = (-2 * p) ** n * boys(n, p * R2_PC)
    
    if t_max >= 1:
        # (n, t=1, u=0, v=0)
        for n in range(0, n_max):
                R[n, 1, 0, 0] = R_PC[0] * R[n+1, 0, 0, 0]

        # (n, t>1, u=0, v=0)
        for t in range(2, t_max + 1):
            for n in range(n_max + 1 - t):
                R[n, t, 0, 0] = R_PC[0] * R[n+1, t-1, 0, 0] + (t-1) * R[n+1, t-2, 0, 0]
    
    if u_max >= 1:
        # (n, t, u=1, v=0)
        for t in range(0, t_max + 1):
            for n in range(0, n_max - t):
                R[n, t, 1, 0] = R_PC[1] * R[n+1, t, 0, 0]

        # (n, t, u>1, v=0)
        for u in range(2, u_max + 1):
            for t in range(0, t_max + 1):
                for n in range(0, n_max + 1 - u - t):
                    R[n, t, u, 0] = R_PC[1] * R[n+1, t, u-1, 0] + (u-1) * R[n+1, t, u-2, 0]
    
    if v_max >=1 :
        # (n, t, u, v=1)
        for u in range(0, u_max + 1):
            for t in range(0, t_max + 1):
                for n in range(0, n_max - u - t):
                    R[n, t, u, 1] = R_PC[2] * R[n+1, t, u, 0]

        # (n, t, u, v>1)
        for v in range(2, v_max + 1):
            for u in range(0, u_max + 1):
                for t in range(0, t_max + 1):
                    for n in range(0, n_max + 1 - v - u - t):
                        R[n, t, u, v] = R_PC[2] * R[n+1, t, u, v-1] + (v-1) * R[n+1, t, u, v-2]
                    
    return R

def _coulomb_attraction_1d_nucleus(E, R, Z_C, a, b, L_A, L_B):
    p = a + b
    Theta = 0
    for t in range(L_A[0] + L_B[0] + 1):
        for u in range(L_A[1] + L_B[1] + 1):
            for v in range(L_A[2] + L_B[2] + 1):
                Theta += E[0, t, L_A[0], L_B[0]] * E[1, u, L_A[1], L_B[1]] * E[2, v, L_A[2], L_B[2]] * R[0, t, u, v]
    return -2 * np.pi / p * Z_C * Theta 

def coulomb_attraction(E, a, b, R_A, R_B, L_A, L_B, R_Cs, Z_Cs):
    n_A = gaussian_norm_coef(a, L_A.sum())
    n_B = gaussian_norm_coef(b, L_B.sum())
    V = 0
    p = a + b
    R_P = (a * R_A + b * R_B) / p
    for R_C, Z_C in zip(R_Cs, Z_Cs):
        R = coulomb_auxiliary_integrals(p, R_P, R_C, t_max=2, u_max=2, v_max=2)
        V += _coulomb_attraction_1d_nucleus(E, R, Z_C, a, b, L_A, L_B)
    return n_A * n_B * V

def coulomb_repulsion(E1, E2, a, b, c, d, R_A, R_B, R_C, R_D, L_A, L_B, L_C, L_D):
    n_A = gaussian_norm_coef(a, L_A.sum())
    n_B = gaussian_norm_coef(b, L_B.sum())
    n_C = gaussian_norm_coef(c, L_C.sum())
    n_D = gaussian_norm_coef(d, L_D.sum())
    p = a + b
    q = c + d
    alpha = p * q / (p + q)
    R_P = (a * R_A + b * R_B) / p
    R_Q = (c * R_C + d * R_D) / q
    L_ABCD = L_A + L_B + L_C + L_D
    R = coulomb_auxiliary_integrals(alpha, R_P, R_Q, t_max=L_ABCD[0], u_max=L_ABCD[1], v_max=L_ABCD[2])
    V = 0
    for t1 in range(L_A[0] + L_B[0] + 1):
        for u1 in range(L_A[1] + L_B[1] + 1):
            for v1 in range(L_A[2] + L_B[2] + 1):
                for t2 in range(L_C[0] + L_D[0] + 1):
                    for u2 in range(L_C[1] + L_D[1] + 1):
                        for v2 in range(L_C[2] + L_D[2] + 1):
                            V += E1[0, t1, L_A[0], L_B[0]] * E1[1, u1, L_A[1], L_B[1]] * E1[2, v1, L_A[2], L_B[2]] * \
                                 E2[0, t2, L_C[0], L_D[0]] * E2[1, u2, L_C[1], L_D[1]] * E2[2, v2, L_C[2], L_D[2]] * \
                                 (-1)**(t2+u2+v2) * R[0, t1+t2, u1+u2, v1+v2]
    return n_A * n_B * n_C * n_D * 2 * (np.pi) ** (5/2) / p / q / np.sqrt(p + q) * V

In [24]:
S = np.full((n_orbitals, n_orbitals), np.nan)
T = np.full((n_orbitals, n_orbitals), np.nan)
V1 = np.full((n_orbitals, n_orbitals), np.nan)

for ao_A in range(n_orbitals):
    for ao_B in range(n_orbitals):

        a, b = ao_exps[ao_A], ao_exps[ao_B]
        d_A, d_B = ao_coefs[ao_A], ao_coefs[ao_B]
        R_A, R_B = Rs[ao2nuc[ao_A]], Rs[ao2nuc[ao_B]]
        L_A, L_B = angular_vector(ao_types[ao_A]), angular_vector(ao_types[ao_B])

        S_AB = np.zeros((a.size, b.size))
        T_AB = np.zeros((a.size, b.size))
        V1_AB = np.zeros((a.size, b.size))
        for i in range(a.size):
            for j in range(b.size):
                E = overlap_expansion_coefs(a[i], b[j], R_A, R_B, 3, 3)
                S_AB[i, j] = d_A[i] * d_B[j] * overlap(E, a[i], b[j], L_A, L_B)
                T_AB[i, j] = d_A[i] * d_B[j] * kinetic(E, a[i], b[j], L_A, L_B)
                V1_AB[i, j] = d_A[i] * d_B[j] * coulomb_attraction(E, a[i], b[j], R_A, R_B, L_A, L_B, Rs, Zs)
                
        S[ao_A, ao_B] = S_AB.sum()
        T[ao_A, ao_B] = T_AB.sum()
        V1[ao_A, ao_B] = V1_AB.sum()

Es = {}
for ao_A in range(n_orbitals):
    for ao_B in range(n_orbitals):
        a, b = ao_exps[ao_A], ao_exps[ao_B]
        R_A, R_B = Rs[ao2nuc[ao_A]], Rs[ao2nuc[ao_B]]
        for i in range(a.size):
            for j in range(b.size):
                Es[(ao_A, ao_B, i, j)] = overlap_expansion_coefs(a[i], b[j], R_A, R_B, 2, 2)

V2 = np.full((n_orbitals, n_orbitals, n_orbitals, n_orbitals), np.nan)
# Pre-screen electron-electron integrals
for ao_A in range(n_orbitals):
    for ao_B in range(n_orbitals):
        if np.isnan(V2[ao_A, ao_B, ao_A, ao_B]):
            a, b = ao_exps[ao_A], ao_exps[ao_B]
            d_A, d_B = ao_coefs[ao_A], ao_coefs[ao_B]
            R_A, R_B = Rs[ao2nuc[ao_A]], Rs[ao2nuc[ao_B]]
            L_A, L_B = angular_vector(ao_types[ao_A]), angular_vector(ao_types[ao_B])
            V2_ABAB = np.zeros((a.size, b.size, a.size, b.size))
            for i in range(a.size):
                for j in range(b.size):
                    for k in range(a.size):
                        for l in range(b.size):
                            E1 = Es[(ao_A, ao_B, i, j)]
                            E2 = Es[(ao_A, ao_B, k, l)]
                            V2_ABAB[i, j, k, l] = d_A[i] * d_B[j] * d_A[k] * d_B[l] * coulomb_repulsion(E1, E2, a[i], b[j], a[k], b[l], R_A, R_B, R_A, R_B, L_A, L_B, L_A, L_B) 
            val = V2_ABAB.sum()
            V2[ao_A, ao_B, ao_A, ao_B] = val
            V2[ao_B, ao_A, ao_B, ao_A] = val
            V2[ao_B, ao_A, ao_A, ao_B] = val
            V2[ao_A, ao_B, ao_B, ao_A] = val            

thres = 1e-7
screened = 0
for ao_A in range(n_orbitals):
    for ao_B in range(n_orbitals):
        for ao_C in range(n_orbitals):
            for ao_D in range(n_orbitals):
                if np.isnan(V2[ao_A, ao_B, ao_C, ao_D]):
                    
                    # Prescreen integral value
                    test = np.sqrt(V2[ao_A, ao_B, ao_A, ao_B]) * np.sqrt(V2[ao_C, ao_D, ao_C, ao_D])
                    if test <= thres:
                        val = 0
                        screened += 1
                    else:
                        a, b, c, d = ao_exps[ao_A], ao_exps[ao_B], ao_exps[ao_C], ao_exps[ao_D]
                        d_A, d_B, d_C, d_D = ao_coefs[ao_A], ao_coefs[ao_B], ao_coefs[ao_C], ao_coefs[ao_D]
                        R_A, R_B, R_C, R_D = Rs[ao2nuc[ao_A]], Rs[ao2nuc[ao_B]], Rs[ao2nuc[ao_C]], Rs[ao2nuc[ao_D]]
                        L_A, L_B, L_C, L_D = angular_vector(ao_types[ao_A]), angular_vector(ao_types[ao_B]), angular_vector(ao_types[ao_C]), angular_vector(ao_types[ao_D])
                        V2_ABCD = np.zeros((a.size, b.size, c.size, d.size))
                        for i in range(a.size):
                            for j in range(b.size):
                                for k in range(c.size):
                                    for l in range(d.size):
                                        E1 = Es[(ao_A, ao_B, i, j)]
                                        E2 = Es[(ao_C, ao_D, k, l)]
                                        V2_ABCD[i, j, k, l] = d_A[i] * d_B[j] * d_C[k] * d_D[l] * coulomb_repulsion(E1, E2, a[i], b[j], c[k], d[l], R_A, R_B, R_C, R_D, L_A, L_B, L_C, L_D) 
                        val = V2_ABCD.sum()
                    V2[ao_A, ao_B, ao_C, ao_D] = val
                    V2[ao_C, ao_D, ao_A, ao_B] = val
                    V2[ao_B, ao_A, ao_D, ao_C] = val
                    V2[ao_D, ao_C, ao_B, ao_A] = val
                    V2[ao_B, ao_A, ao_C, ao_D] = val
                    V2[ao_D, ao_C, ao_A, ao_B] = val
                    V2[ao_A, ao_B, ao_D, ao_C] = val
                    V2[ao_C, ao_D, ao_B, ao_A] = val
                    #print(ao_A, ao_B, ao_C, ao_D)

In [25]:
# Nuclear repulsion energy
V_NN = 0
for i in range(n_nuclei):
    for j in range(i+1, n_nuclei):
        V_NN += Zs[i] * Zs[j] / np.sqrt(np.sum((Rs[i] - Rs[j]) ** 2))

In [26]:
V_NN

13.497304462033398

In [27]:
eig_S, U = np.linalg.eigh(S)
eig_S_sqrt = eig_S ** (-1/2) * np.eye(len(eig_S))
X = U @ eig_S_sqrt @ U.conj().T
H = T + V1

In [28]:
F = H.copy()
F_ = X.conj().T @ F @ X 
eps, C_ = np.linalg.eigh(F_)
C = X @ C_
P = np.zeros((n_orbitals, n_orbitals))
for mu in range(n_orbitals):
    for nu in range(n_orbitals):
        for m in range(N // 2):
            P[mu, nu] += C[mu, m] * C[nu, m]
G = np.zeros((n_orbitals, n_orbitals))
for ao1 in range(n_orbitals):
    for ao2 in range(n_orbitals):
        for ao3 in range(n_orbitals):
            for ao4 in range(n_orbitals):
                G[ao1, ao2] += (P[ao3, ao4] * (2 * V2[ao1, ao2, ao4, ao3] - V2[ao1, ao3, ao4, ao2]))
F = H + G

In [29]:
F = H.copy()
# F = np.zeros((n_orbitals, n_orbitals))
# for mu in range(n_orbitals):
#     for nu in range(n_orbitals):
#         F[mu, nu] = 1.75 * S[mu, nu] * (H[mu, nu] + H[nu, nu]) / 2
P = np.zeros((n_orbitals, n_orbitals))
P_prev = np.full((n_orbitals, n_orbitals), np.inf)
i = 0
while np.max(np.abs(P - P_prev)) > 1e-8:
    F_ = X.conj().T @ F @ X 
    eps, C_ = np.linalg.eigh(F_)
    C = X @ C_
    P_prev = P
    P = np.zeros((n_orbitals, n_orbitals))
    for mu in range(n_orbitals):
        for nu in range(n_orbitals):
            for m in range(N // 2):
                P[mu, nu] += C[mu, m] * C[nu, m]
    G = np.zeros((n_orbitals, n_orbitals))
    for ao1 in range(n_orbitals):
        for ao2 in range(n_orbitals):
            for ao3 in range(n_orbitals):
                for ao4 in range(n_orbitals):
                    G[ao1, ao2] += (P[ao3, ao4] * (2 * V2[ao1, ao2, ao4, ao3] - V2[ao1, ao3, ao4, ao2]))
    F = H + G
    E0 = np.sum(P * (H + F))
    E_tot = E0 + V_NN
    print(i, E0, E_tot)
    
    i += 1

0 -49.58075303527532 -36.083448573241924
1 -53.06181788117689 -39.564513419143495
2 -53.21914078576 -39.7218363237266
3 -53.22399745997904 -39.72669299794564
4 -53.22414981544592 -39.72684535341252
5 -53.224154619528264 -39.726850157494866
6 -53.224154773175286 -39.72685031114189
7 -53.22415477821302 -39.72685031617962
8 -53.22415477838563 -39.726850316352234
9 -53.22415477839198 -39.72685031635858
10 -53.22415477839223 -39.726850316358835
11 -53.22415477839224 -39.72685031635884
12 -53.22415477839225 -39.72685031635885
13 -53.22415477839226 -39.72685031635886


In [17]:
eps

array([-11.0698062 ,  -0.95626083,  -0.50657467,  -0.50657467,
        -0.50657467,   0.43292629,   0.72149175,   0.72149175,
         0.72149175])

In [63]:
# U = np.array([
#         [2**(-1/2), 2**(-1/2)],
#         [2**(-1/2), -2**(-1/2)]
#     ])
# s = np.diag([(1 + S[0, 1]) ** (-1/2),(1 - S[0, 1]) ** (-1/2)]) 
# X = U @ s

In [40]:
V2

array([[[[1.30714751, 0.43727805],
         [0.43727805, 0.60570147]],

        [[0.43727805, 0.1772667 ],
         [0.1772667 , 0.31179381]]],


       [[[0.43727805, 0.1772667 ],
         [0.1772667 , 0.31179381]],

        [[0.60570147, 0.31179381],
         [0.31179381, 0.77460593]]]])

In [23]:
def calc_overlap_matrix(a, b, R_A, R_B, l_A_max=0, l_B_max=0):
    s = np.zeros((3, l_A_max + 2, l_B_max + 1))
    
    for i in range(3):
        
        # Base Case
        s[i, 0, 0] = 1
        R_P_i = (a * R_A[i] + b * R_B[i]) / (a + b)
        s[i, 1, 0] = R_P_i - R_A[i]
        
        # Recurrence relation
        for l_A in range(2, l_A_max + 2):
            s[i, l_A, 0] = (R_P_i - R_A[i]) * s[i, l_A-1, 0] + (l_A - 1) / (2 * (a + b)) * s[i, l_A-2, 0]
        
        # Transfer equation
        for l_B in range(1, l_B_max + 1):
            for l_A in range(0, l_A_max + 2):
                s[i, l_A-1, l_B] = s[i, l_A, l_B-1] + (R_A[i] - R_B[i]) * s[i, l_A-1, l_B - 1] 
            
    return s[:, :-1, :]



In [24]:
def gaussian_norm_coef(a, l_A):
    if l_A == 0:
        return (2 * a / np.pi) ** (3/4)
    elif l_A == 1:
        return (128 * a ** 5 / np.pi ** 3) ** (1/4)
    else:
        raise NotImplementedError('Input l_A must be an integer <= 1.')

def overlap(s, a, b, R_A, R_B, L_A=np.array([0, 0, 0]), L_B=np.array([0, 0, 0])):
    p = a + b
    R2_AB = np.sum((R_a - R_b) ** 2)
    E_AB = np.exp(-a * b / p * R2_AB)
    n_A = gaussian_norm_coef(a, L_A.sum())
    n_B = gaussian_norm_coef(b, L_B.sum())
    S = n_A * n_B * (np.pi / p) ** (3/2) * E_AB * s[0, L_A[0], L_B[0]] * s[1, L_A[1], L_B[1]] * s[2, L_A[2], L_B[2]]
    return S

In [376]:
s = calc_overlap_matrix(a, b, R_a, R_b, 1, 1)

In [378]:
def angular_vector(ao_type):
    if ao_type == 'S':
        return np.array([0, 0, 0])
    elif ao_type == 'Px':
        return np.array([1, 0, 0])
    elif ao_type == 'Py':
        return np.array([0, 1, 0])
    elif ao_type == 'Pz':
        return np.array([0, 0, 1])
    else:
        raise NotImplementedError()

In [410]:
S = np.zeros((n_orbitals, n_orbitals))

for ao_A in range(n_orbitals):
    for ao_B in range(n_orbitals):
        
        a, b = ao_exps[ao_A], ao_exps[ao_B]
        d_A, d_B = ao_coefs[ao_A], ao_coefs[ao_B]
        R_A, R_B = Rs[ao_A], Rs[ao_B]
        L_A, L_B = angular_vector(ao_types[ao_A]), angular_vector(ao_types[ao_B])
        
        S_AB = np.zeros((a.size, b.size))
        for i in range(a.size):
            for j in range(b.size):
                s = calc_overlap_matrix(a[i], b[j], R_a, R_b, L_A.sum(), L_B.sum())
                S_AB[i, j] = d_A[i] * d_B[j] * overlap(s, a[i], b[j], R_A, R_B, L_A, L_B)
                
        S[ao_A, ao_B] = S_AB.sum()

In [415]:
S.round(4)

array([[ 0.5028,  0.5028,  0.05  ,  0.454 ,  0.    ,  0.2928, -0.2455],
       [ 0.5028,  0.5028,  0.05  ,  0.454 ,  0.    ,  0.2928, -0.2455],
       [ 0.05  ,  0.05  ,  0.    ,  0.0459,  0.    ,  0.0584, -0.049 ],
       [ 0.454 ,  0.454 ,  0.0459,  0.4198,  0.    ,  0.3257, -0.2732],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.2573,  0.    ,  0.    ],
       [-0.2928, -0.2928, -0.0584, -0.3257,  0.    , -0.0847,  0.2869],
       [ 0.2455,  0.2455,  0.049 ,  0.2732,  0.    ,  0.2869,  0.0167]])

In [372]:
a, b = ao_exps[0, 0], ao_exps[6, 0]
R_a, R_b = Rs[0], Rs[6]
overlap(s, a, b, R_a, R_B, np.array([0, 0, 0]), np.array([0, 0, 1]))

-0.0017123475858432068

In [289]:
def overlap(a, b, R_A, R_B, i_A=np.array([0, 0, 0]), i_B=np.array([0, 0, 0])):
    '''
    Evaluates all pairs of normalized overlap integrals for two sets of Gaussians, given exponents (`a`, `b`), 3D centers (`R_A`, `R_B`), and 3D Cartesian angular values (`i_A`, `i_B`).  This function supports s and p orbitals only; that is, `i_A` and `i_B` may each have a maximum of one non-zero component.  For example, a p_x orbital would be expressed using `i_A = np.array([1, 0, 0])`.    
    '''
    a, b = np.meshgrid(a, b, indexing='ij')
    ab = a * b / (a + b)
    l_A = np.sum(i_A)
    l_B = np.sum(i_B)
    R2_AB = np.sum((R_B - R_A) ** 2)
    
    if l_A == 0:
        n_A = (2 * a / np.pi) ** (3/4)
    elif l_A == 1:
        n_A = (128 * a ** 5 / np.pi ** 3) ** (1/4)
    else:
        raise ValueError('Input error for i_A.')
        
    if l_B == 0:
        n_B = (2 * b / np.pi) ** (3/4)
    elif l_B:
        n_B = (128 * b ** 5 / np.pi ** 3) ** (1/4)
    else:
        raise ValueError('Input error for i_B.')
        
    R_P = np.zeros((len(a), len(b), 3))
    for i in range(len(a)):
        for j in range(len(b)):
            R_P[i, j, :] = (a[i, i] * R_A + b[j, j] * R_B) / (a[i, i] + b[j, j])
    R_AP = R_P - R_A
    R_BP = R_P - R_B
        
    # Calculate base case overlap integral
    S_00 = (np.pi / (a + b)) ** (3/2) * np.exp(-ab * R2_AB)
    
    if l_A == 0 and l_B == 0:
        return n_A * n_B * S_00
    elif l_A == 1 and l_B == 0:
        return  n_A * n_B * R_AP[:, :, np.argmax(i_A)] * S_00
    elif l_A == 0 and l_B == 1:
        return  n_A * n_B * R_BP[:, :, np.argmax(i_B)] * S_00
    elif l_A == 1 and l_B == 1:
        m_A = np.argmax(i_A)
        m_B = np.argmax(i_B)
        if m_A != m_B:
            return n_A * n_B * R_AP[:, :, m_A] * R_BP[:, :, m_B] * S_00
        else:
            return n_A * n_B * (R_AP[:, :, m_A] * R_BP[:, :, m_B] * S_00 + 1 / (2 * (a + b)) * S_00)

In [315]:
S = np.zeros((n_orbitals, n_orbitals))
for ao1 in range(n_orbitals):
    for ao2 in range(n_orbitals):
        
        a, b = ao_exps[ao1], ao_exps[ao2]
        R_A, R_B = Rs[ao1], Rs[ao2]
        d_A, d_B = np.meshgrid(ao_coefs[ao1], ao_coefs[ao2], indexing='ij')
        
        if ao_types[ao1] == 'S':
            i_A = np.array([0, 0, 0])
        elif ao_types[ao1] == 'Px':
            i_A = np.array([1, 0, 0])
        elif ao_types[ao1] == 'Py':
            i_A = np.array([0, 1, 0])
        elif ao_types[ao1] == 'Pz':
            i_A = np.array([0, 0, 1])
            
        if ao_types[ao2] == 'S':
            i_B = np.array([0, 0, 0])
        elif ao_types[ao2] == 'Px':
            i_B = np.array([1, 0, 0])
        elif ao_types[ao2] == 'Py':
            i_B = np.array([0, 1, 0])
        elif ao_types[ao2] == 'Pz':
            i_B = np.array([0, 0, 1])
            
        S[ao1, ao2] = np.sum(d_A * d_B * overlap(a, b, R_A, R_B, i_A, i_B))

CPU times: user 18.6 ms, sys: 2.54 ms, total: 21.1 ms
Wall time: 21 ms


In [316]:
S.round(3)

array([[ 1.   ,  0.251,  0.05 ,  0.454,  0.   ,  0.293, -0.246],
       [ 0.251,  1.   ,  0.05 ,  0.454,  0.   , -0.293, -0.246],
       [ 0.05 ,  0.05 ,  1.   ,  0.237,  0.   ,  0.   , -0.   ],
       [ 0.454,  0.454,  0.237,  1.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ],
       [ 0.293, -0.293,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [-0.246, -0.246, -0.   ,  0.   ,  0.   ,  0.   ,  1.   ]])

In [306]:
np.sum(ao_coefs[5].reshape(-1, 1) * ao_coefs[6] * overlap(ao_exps[5], ao_exps[6], Rs[5], Rs[6], np.array([0, 1, 0]), np.array([0, 0, 1])))

0.0

In [ ]:
S = np.zeros((n_orbitals, n_orbitals))
T = np.zeros((n_orbitals, n_orbitals))
V = np.zeros((n_orbitals, n_orbitals))
for ao1 in [0]:
    for ao2 in [6]:

        a, b = np.meshgrid(ao_exps[ao1], ao_exps[ao2], indexing='ij')
        ab = a * b / (a + b)
        d_A, d_B = np.meshgrid(ao_coefs[ao1], ao_coefs[ao2], indexing='ij') 
        n_A, n_B = (2 * a / np.pi) ** (3/4), (2 * b / np.pi) ** (3/4)
        n_AB = n_A * n_B * (np.pi / (a + b)) ** (3/2)
        R_A, R_B = Rs[ao1], Rs[ao2]
        R2_AB = np.sum((R_A - R_B) ** 2)
        
         # Calculate overlap integral
        S[ao1, ao2] = np.sum(d_A * d_B * n_AB * np.exp(-ab * R2_AB))
        
        # Calculate kinetic energy
        T[ao1, ao2] = np.sum(d_A * d_B * n_AB * ab * (3 - 2 * ab * R2_AB) * np.exp(-ab * R2_AB))
        
        # Calculate potential energies
        R_P = np.zeros((3, 3, 3))
        for i in range(3):
            for j in range(3):
                R_P[i, j, :] = (ao_exps[ao1, i] * R_A + ao_exps[ao2, j] * R_B) / \
                                (ao_exps[ao1, i] + ao_exps[ao2, j])
        for C in range(n_nuclei):
            R_C = Rs[C]
            R_PC = R_P - np.tile(R_C, (3, 3, 1))
            R2_PC = np.sum(R_PC ** 2, 2)
            V[ao1, ao2] += np.sum(d_A * d_B * n_A * n_B * -2 * np.pi / (a + b) * np.exp(-ab * R2_AB) * \
                Zs[C] * f0((a + b) * R2_PC))
        
H = T + V

In [261]:
z = (R_P - R_B)[:, :, -1]

In [269]:
n_A, n_B = (2 * a / np.pi) ** (3/4), (128 * b ** 5 / np.pi ** 3) ** (1/4)
n_AB = n_A * n_B * (np.pi / (a + b)) ** (3/2)

In [270]:
np.sum(z * d_A * d_B * n_AB * np.exp(-ab * R2_AB))

-0.245538187706151

In [215]:
n_B

array([[10.74583258,  1.73374402,  0.42581899],
       [10.74583258,  1.73374402,  0.42581899],
       [10.74583258,  1.73374402,  0.42581899]])

In [149]:
S.round(2)

array([[1.  , 0.25, 0.05, 0.45, 0.38, 0.38, 0.38],
       [0.25, 1.  , 0.05, 0.45, 0.38, 0.38, 0.38],
       [0.05, 0.05, 1.  , 0.24, 0.47, 0.47, 0.47],
       [0.45, 0.45, 0.24, 1.  , 0.99, 0.99, 0.99],
       [0.38, 0.38, 0.47, 0.99, 1.1 , 1.1 , 1.1 ],
       [0.38, 0.38, 0.47, 0.99, 1.1 , 1.1 , 1.1 ],
       [0.38, 0.38, 0.47, 0.99, 1.1 , 1.1 , 1.1 ]])

In [32]:
ele_ele_repulsion = np.empty((n_orbitals, n_orbitals, n_orbitals, n_orbitals))
for ao1 in range(n_orbitals):
    for ao2 in range(n_orbitals):
        for ao3 in range(n_orbitals):
            for ao4 in range(n_orbitals):
                
                ao1x, ao2x, ao3x, ao4x = np.meshgrid(ao_exps[ao1], ao_exps[ao2], 
                                     ao_exps[ao3], ao_exps[ao4], indexing='ij')
                ao1c, ao2c, ao3c, ao4c = np.meshgrid(ao_coefs[ao1], ao_coefs[ao2], 
                                                     ao_coefs[ao3], ao_coefs[ao4], indexing='ij')
                ao1n, ao2n, ao3n, ao4n = np.meshgrid((2 * ao_exps[ao1] / np.pi) ** (3/4), 
                                                     (2 * ao_exps[ao2] / np.pi) ** (3/4), 
                                                     (2 * ao_exps[ao3] / np.pi) ** (3/4), 
                                                     (2 * ao_exps[ao4] / np.pi) ** (3/4), 
                                                     indexing='ij') 
                ao1R, ao2R, ao3R, ao4R = Rs[[ao1, ao2, ao3, ao4]]
                R1_R2_diff_sq = np.dot(ao2R - ao1R, ao2R - ao1R)
                R3_R4_diff_sq = np.dot(ao4R - ao3R, ao4R - ao3R)
                ao12R = np.empty((3, 3, 3))
                for i in range(3):
                    for j in range(3):
                        ao12R[i, j, :] = (ao_exps[ao1, i] * Rs[ao1] + ao_exps[ao2, j] * Rs[ao2]) / \
                            (ao_exps[ao1, i] + ao_exps[ao2, j])

                ao34R = np.empty((3, 3, 3))
                for i in range(3):
                    for j in range(3):
                        ao34R[i, j, :] = (ao_exps[ao3, i] * Rs[ao3] + ao_exps[ao4, j] * Rs[ao4]) / \
                            (ao_exps[ao3, i] + ao_exps[ao4, j])

                R12_R34_diff_sq = np.empty((3, 3, 3, 3))
                for i in range(3):
                    for j in range(3):
                        for k in range(3):
                            for l in range(3):
                                R12_R34_diff_sq[i, j, k, l] = np.dot(ao34R[k, l] - ao12R[i, j], 
                                                                     ao34R[k, l] - ao12R[i, j])

                ele_ele_repulsion_tmp = ao1n * ao2n * ao3n * ao4n * 2 * np.pi ** (5/2) / \
                    ((ao1x + ao2x) * (ao3x + ao4x) * (ao1x + ao2x + ao3x + ao4x) ** (1/2)) * \
                    np.exp(-ao1x * ao2x / (ao1x + ao2x) * R1_R2_diff_sq - 
                           ao3x * ao4x / (ao3x + ao4x) * R3_R4_diff_sq) * \
                    f0((ao1x + ao2x) * (ao3x + ao4x) / (ao1x + ao2x + ao3x + ao4x) * R12_R34_diff_sq)

                ele_ele_repulsion[ao1, ao2, ao3, ao4] = np.sum(ao1c * ao2c * ao3c * ao4c * ele_ele_repulsion_tmp)


In [33]:
ele_ele_repulsion

array([[[[0.77460593, 0.44410765],
         [0.44410765, 0.56967592]],

        [[0.44410765, 0.29702853],
         [0.29702853, 0.44410765]]],


       [[[0.44410765, 0.29702853],
         [0.29702853, 0.44410765]],

        [[0.56967592, 0.44410765],
         [0.44410765, 0.77460593]]]])

In [44]:
U = np.array([
        [2**(-1/2), 2**(-1/2)],
        [2**(-1/2), -2**(-1/2)]
    ])
s = np.diag([(1 + S[0, 1]) ** (-1/2),(1 - S[0, 1]) ** (-1/2)]) 
X = U @ s

In [45]:
F = H.copy()
P = np.zeros((N, N))
P_prev = np.full((N, N), np.inf)
i = 0
while np.max(np.abs(P - P_prev)) > 1e-6:
    F_ = X.conj().T @ F @ X 
    eps, C_ = np.linalg.eigh(F_)
    C = X @ C_
    P_prev = P
    P = 2 * C[:, :int(N/2)] @ C[:, :int(N/2)].conj().T
    G = np.zeros((N, N))
    for ao1 in range(N):
        for ao2 in range(N):
            for ao3 in range(N):
                for ao4 in range(N):
                    G[ao1, ao2] += np.sum(P[ao3, ao4] * (ele_ele_repulsion[ao1, ao2, ao4, ao3] - \
                                                         1/2 * ele_ele_repulsion[ao1, ao3, ao4, ao2]))
    F = H + G
    E0 = 1/2 * np.sum(P * (H + F))
    print(i, P[0, 0], P[1, 0], P[1, 1], E0)
    
    i += 1

NameError: name 'ele_ele_repulsion' is not defined

In [38]:
E0 + 1 / 1.4

-1.1167143186700597

In [480]:
F = H.copy()

In [482]:
F

array([[-2.6527429 , -1.34720354],
       [-1.34720354, -1.73182566]])

In [483]:
F_ = X.conj().T @ F @ X 

In [484]:
eps, C_ = np.linalg.eigh(F_)

In [487]:
C_

array([[-0.91044534,  0.4136294 ],
       [-0.4136294 , -0.91044534]])

In [488]:
C = X @ C_

In [489]:
P = 2 * C[:, :int(N/2)] @ C[:, :int(N/2)].conj().T

In [490]:
P

array([[1.72662644, 0.25985178],
       [0.25985178, 0.03910687]])

In [491]:
G = np.zeros((N, N))
for ao1 in range(N):
    for ao2 in range(N):
        for ao3 in range(N):
            for ao4 in range(N):
                G[ao1, ao2] += np.sum(P[ao3, ao4] * (ele_ele_repulsion[ao1, ao2, ao4, ao3] - \
                                                     1/2 * ele_ele_repulsion[ao1, ao3, ao4, ao2]))

In [492]:
G

array([[1.26232841, 0.37400312],
       [0.37400312, 0.98895078]])

In [494]:
H + G

array([[-1.39041449, -0.97320042],
       [-0.97320042, -0.74287488]])